In [ ]:
data Op = Plus | Minus | Mul | Div | Pow
    deriving (Eq, Show)
    
data SymbolicManip a =
    Number a
  | Symbol S
  | BinaryArith Op (SymbolicManip a) (SymbolicManip a)
  | UnaryArith String (SymbolicManip a)
    deriving (Eq, Show)

In [ ]:
instance Num a => Num (SymbolicManip a) where
    a + b = Arith Plus a b
    a - b = Arith Minus a b
    a * b = Arith Mul a b
    negate a = Arith Mul (Number (-1)) a
    abs a = error "abs is unimplemented"
    signum a = error "signum is unimplemented"
    fromInteger i = Number (fromInteger i)

In [ ]:
five = Number 5
ten = Number 10
m = five * ten

In [ ]:
print ((5 * 10) :: SymbolicManip Int)